# Credit Risk Scorecard Model Validation

## Before you begin
To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also create a documentation project on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
# %pip install --upgrade validmind

## Initialize the client library
In a browser, go to the Client Integration page of your documentation project and click Copy to clipboard next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

This step requires a documentation project. Learn how you can create one.

Next, replace this placeholder with your own code snippet:

In [2]:

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
  project = "cllaz74gb067dszy6donpqm98"
)
  

2023-08-21 14:01:54,797 - INFO(validmind.api_client): Connected to ValidMind. Project: [9] Credit Risk Scorecard - Initial Validation (cllaz74gb067dszy6donpqm98)


## Introduction

The **Credit risk Scorecard** model created from the Lending Club dataset is instrumental in computing the Probability of Default (PD), a key factor in ECL calculations. This scorecard assesses several credit characteristics of potential borrowers, like their credit history, income, outstanding debts, and more, each of which is assigned a specific score. By combining these scores, we derive a total score for each borrower, which translates into an estimated Point-in-Time (PiT) PD. The PiT PD reflects the borrower's likelihood of default at a specific point in time, accounting for both current and foreseeable future conditions.

Additionally, for a holistic view of credit risk, it's essential to estimate the Lifetime PD. The Lifetime PD, as the name suggests, predicts the borrower's likelihood of default throughout the life of the exposure, taking into account potential future changes in the economic and financial conditions.

## Setup

### Import Libraries

In [3]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

from IPython.display import HTML
from notebooks.probability_of_default.helpers.Developer import Developer
from notebooks.probability_of_default.helpers.scorecard_model import *
from notebooks.probability_of_default.helpers.model_development import *

# Visualization imports
%matplotlib inline

### Input Parameters

In [4]:
default_column = "default"

### Load Datasets and Models

In [5]:
developer = Developer()
scorecard = developer.load_objects_from_pickle("datasets/scorecard_data_and_models.pkl")

df_raw = scorecard["df_raw"]
df_prepared = scorecard["df_prepared"]
df_train = scorecard["df_train"]
df_train_feature_selection = scorecard["df_train_feature_selection"]

df_train_feature_eng = scorecard["df_train_feature_eng"]
df_test_feature_eng = scorecard["df_test_feature_eng"]

model_fit_final = scorecard["model_fit_final"]

INFO: Loaded 7 objects from datasets/scorecard_data_and_models.pkl


### Create ValidMind Datasets and Model

In [6]:
from validmind.vm_models.test_context import TestContext

vm_df_raw = vm.init_dataset(dataset=df_raw, target_column=default_column)
vm_df_prepared = vm.init_dataset(dataset=df_prepared, target_column=default_column)
vm_df_train_feature_selection = vm.init_dataset(dataset=df_train_feature_selection, target_column=default_column)

vm_df_train_feature_eng = vm.init_dataset(dataset=df_train_feature_eng, target_column=default_column)
vm_df_test_feature_eng = vm.init_dataset(dataset=df_test_feature_eng, target_column=default_column)

vm_model_fit_final = vm.init_model(
    model = model_fit_final, 
    train_ds=vm_df_train_feature_eng, 
    test_ds=vm_df_test_feature_eng)


test_context_raw = TestContext(dataset=vm_df_raw)
test_context_prepared = TestContext(dataset=vm_df_prepared)
test_context_train_feature_selection = TestContext(dataset=vm_df_train_feature_selection)

test_context_models_fit_final = TestContext(models = [vm_model_fit_final])
test_context_model_fit_final = TestContext(model = [vm_model_fit_final])

2023-08-21 14:01:55,137 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...
2023-08-21 14:01:56,044 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...
2023-08-21 14:01:57,089 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...
2023-08-21 14:01:57,708 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...
2023-08-21 14:01:58,672 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...


## Model validation

### Data Description

In [7]:
from validmind.tests.data_validation.DescriptiveStatistics import DescriptiveStatistics

metric = DescriptiveStatistics(test_context_raw, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [8]:
from validmind.tests.data_validation.MissingValuesBarPlot import MissingValuesBarPlot

params = {"threshold": 80,
          "fig_height": 1100}

metric = MissingValuesBarPlot(test_context_raw, params)
metric.run()
await metric.result.log()
metric.result.show()

### Data Preparation

In [9]:
from validmind.tests.data_validation.ClassImbalance import ClassImbalance

metric = ClassImbalance(test_context_prepared, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [10]:
from validmind.tests.data_validation.IQROutliersTable import IQROutliersTable

numerical_features = get_numerical_columns(df_prepared)
params = {"num_features": numerical_features,
          "threshold": 1.5
        }

metric = IQROutliersTable(test_context_prepared, params)
metric.run()
await metric.result.log()
metric.result.show()

In [11]:
from validmind.tests.data_validation.IQROutliersBarPlot import IQROutliersBarPlot

numerical_features = get_numerical_columns(df_prepared)
params = {"num_features": numerical_features,
          "threshold": 1.5,
          "fig_width": 500}

metric = IQROutliersBarPlot(test_context_prepared, params)
metric.run()
await metric.result.log()
metric.result.show()

### Exploratory Data Analysis

In [12]:
from validmind.tests.data_validation.TabularNumericalHistograms import TabularNumericalHistograms

metric = TabularNumericalHistograms(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [13]:
from validmind.tests.data_validation.HighCardinality import HighCardinality
metric = HighCardinality(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [14]:
from validmind.tests.data_validation.TabularCategoricalBarPlots import TabularCategoricalBarPlots
metric = TabularCategoricalBarPlots(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [15]:
from validmind.tests.data_validation.TargetRateBarPlots import TargetRateBarPlots

# Configure the metric
params = {
    "default_column": default_column,
    "columns": None
}

metric = TargetRateBarPlots(test_context_train_feature_selection, params=params)
metric.run()
await metric.result.log()
metric.result.show()

The column default is correct and contains only 1 and 0.


In [16]:
from validmind.tests.data_validation.ChiSquaredFeaturesTable import ChiSquaredFeaturesTable

categorical_features = get_categorical_columns(df_train_feature_selection)
params = {"cat_features": categorical_features,
          "p_threshold": 0.05}

metric = ChiSquaredFeaturesTable(test_context_train_feature_selection, params)
metric.run()
await metric.result.log() 
metric.result.show()

In [17]:
from validmind.tests.data_validation.ANOVAOneWayTable import ANOVAOneWayTable

numerical_features = get_numerical_columns(df_train_feature_selection)
params = {"num_features": numerical_features,
          "p_threshold": 0.05}

metric = ANOVAOneWayTable(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

In [18]:
from validmind.tests.data_validation.PearsonCorrelationMatrix import PearsonCorrelationMatrix

params = {"declutter": False,
          "features": None,
          "fontsize": 13}

metric = PearsonCorrelationMatrix(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

In [19]:
from validmind.tests.data_validation.FeatureTargetCorrelationPlot import FeatureTargetCorrelationPlot

params = {"features": None}

metric = FeatureTargetCorrelationPlot(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

### Feature Engineering

In [20]:
from validmind.tests.data_validation.WOEBinTable import WOEBinTable

metric = WOEBinTable(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

Running with breaks_adj: None
Performing binning with breaks_adj: None
[INFO] creating woe binning ...


/Users/juanvalidmind/Library/Caches/pypoetry/virtualenvs/validmind-X_uvMH0R-py3.10/lib/python3.10/site-packages/scorecardpy/condition_fun.py:79: UserWarning:

There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: emp_length)



In [21]:
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

metric = WOEBinTable(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

Running with breaks_adj: {'int_rate': [5, 10, 15]}
Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...


/Users/juanvalidmind/Library/Caches/pypoetry/virtualenvs/validmind-X_uvMH0R-py3.10/lib/python3.10/site-packages/scorecardpy/condition_fun.py:79: UserWarning:

There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: emp_length)



In [22]:
from validmind.tests.data_validation.WOEBinPlots import WOEBinPlots

params = {
    "breaks_adj": {"int_rate": [5,10,15]},
    "fig_height": 500,
}

metric = WOEBinPlots(test_context_train_feature_selection, params=params)
metric.run()
await metric.result.log()
metric.result.show()

[INFO] creating woe binning ...


/Users/juanvalidmind/Library/Caches/pypoetry/virtualenvs/validmind-X_uvMH0R-py3.10/lib/python3.10/site-packages/scorecardpy/condition_fun.py:79: UserWarning:

There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: emp_length)



### Model Training

In [23]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

metric = RegressionCoeffsPlot(test_context_models_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [24]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_models_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

### Model Evaluation

In [25]:
from validmind.tests.model_validation.statsmodels.LogRegressionConfusionMatrix import LogRegressionConfusionMatrix

# Configure test parameters
params = {
    "cut_off_threshold": 0.5,
}

metric = LogRegressionConfusionMatrix(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [26]:
from validmind.tests.model_validation.statsmodels.RegressionROCCurve import RegressionROCCurve

metric = RegressionROCCurve(test_context_model_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [27]:
from validmind.tests.model_validation.statsmodels.GINITable import GINITable

metric = GINITable(test_context_model_fit_final, params=None)
metric.run()
await metric.result.log() 
metric.result.show()

Predicted scores obtained...
Computing AUC...
Computing GINI...
Computing AUC...
Computing KS...
Predicted scores obtained...
Computing AUC...
Computing GINI...
Computing AUC...
Computing KS...


In [28]:
from validmind.tests.model_validation.statsmodels.LogisticRegPredictionHistogram import LogisticRegPredictionHistogram

# Configure test parameters
params = {
    "title": "Histogram of Probability of Default",
}

metric = LogisticRegPredictionHistogram(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [29]:
from validmind.tests.model_validation.statsmodels.LogisticRegCumulativeProb import LogisticRegCumulativeProb

# Configure test parameters
params = {
    "title": "Cumulative Probability of Default",
}

metric = LogisticRegCumulativeProb(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [30]:
from validmind.tests.model_validation.statsmodels.ScorecardHistogram import ScorecardHistogram

# Configure test parameters
params = {
    "target_score": 600,
    "target_odds": 50,
    "pdo": 20,
    "title": "Histogram of Credit Scores",
}

metric = ScorecardHistogram(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()